#### What are the expected reletionships of independent variables with review score?
1. order_items_qty ("+") - if consumer gets more than one item from the same seller, it should mean that he/she knows the quality of the good. Therefore, increase in item quantity should increase review score
2. product_description_lenght ("+") - buyer having more information about buying product should have positive relationship with review score
3. product_photos_qty ("+")
4. delivery_accuracy ("+") - item coming on time or earlier that it was described should have positive relationship with review score
5. order_products_value ("unknown") - more expensive perhaps means better quality or higher expectation; thus, it is hard to predict